## 서울시의 스타벅스 매장의 이름과 주소, 구 이름을 pandas data frame으로 정리

In [475]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

# 스타벅스 페이지 접근
driver = webdriver.Chrome()
driver.get('https://www.starbucks.co.kr/store/store_map.do?disp=locale')
time.sleep(5)

# 설날 팝업 삭제
driver.find_element(By.XPATH, '''/html/body/div[4]/p/a''').click()

# 지역 선택 클릭
driver.find_element(By.CLASS_NAME,'loca_search').click()

time.sleep(2)

# 서울 선택
driver.find_element(By.XPATH, '''//*[@id="container"]/div/form/fieldset/div/section/article[1]/article/article[2]/div[1]/div[2]/ul/li[1]/a''').click()

In [476]:
from bs4 import BeautifulSoup
import pandas as pd

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
print(soup.prettify())

<html lang="ko">
 <head>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no" name="viewport"/>
  <meta content="website" property="og:type"/>
  <meta content="Starbucks" property="og:title"/>
  <meta content="https://www.starbucks.co.kr/" property="og:url"/>
  <meta content="https://image.istarbucks.co.kr/common/img/kakaotalk.png" property="og:image"/>
  <meta content="Starbucks" property="og:description"/>
  <title id="titleJoin">
   Starbucks Korea
  </title>
  <script async="" crossorigin="anonymous" src="https://connect.facebook.net/ko_KR/sdk.js?hash=18377a33d8c6d63e48b7b3e2ae9f4599">
  </script>
  <script id="facebook-jssdk" src="//connect.facebook.net/ko_KR/sdk.js">
  </script>
  <script async="" src="https://www.googletagmanager.com/gtag/js?id=G-WC8Q3C59QP&amp;cx=c&amp;_slc=1" type="text/javascript">
  </script>
  <script async="" src="//

In [477]:
# 구 이름 담긴 guList 만들기
gus = driver.find_elements(By.CLASS_NAME, 'set_gugun_cd_btn')
guList = []
for gu in gus:
    guList.append(gu.text)
guList.pop(0)
guList = [gu for gu in guList if gu != '']
guList
#len(guList)

['강남구',
 '강동구',
 '강북구',
 '강서구',
 '관악구',
 '광진구',
 '구로구',
 '금천구',
 '노원구',
 '도봉구',
 '동대문구',
 '동작구',
 '마포구',
 '서대문구',
 '서초구',
 '성동구',
 '성북구',
 '송파구',
 '양천구',
 '영등포구',
 '용산구']

In [478]:
# 구 이름 담긴 guList 만들기2 -> 스크롤해야하는 지역 추가 
scroll = driver.find_element(By.ID, 'mCSB_2_container')
driver.execute_script('arguments[0].style.top = "-130px";', scroll)

In [479]:
gus2 = driver.find_elements(By.CLASS_NAME, 'set_gugun_cd_btn')
for gu in gus2:
    guList.append(gu.text)
#list(set(guList))

In [480]:
guList = [gu for gu in guList if gu != '']
guList = list(set(guList))

In [481]:
# 구 개수 확인
len(guList)

25

In [482]:
guList

['관악구',
 '강북구',
 '광진구',
 '도봉구',
 '구로구',
 '중랑구',
 '강남구',
 '동작구',
 '성동구',
 '노원구',
 '서초구',
 '종로구',
 '송파구',
 '서대문구',
 '용산구',
 '동대문구',
 '강동구',
 '은평구',
 '양천구',
 '강서구',
 '마포구',
 '성북구',
 '금천구',
 '중구',
 '영등포구']

In [483]:
# 구, 지점명, 주소, 위도, 경도 가져오는 함수 선언
def nameNaddress(gu):
    driver.execute_script("window.scrollTo(0, 0);")
    driver.find_element(By.CSS_SELECTOR, '#container > div > form > fieldset > div > section > article.find_store_cont > article > header.loca_search > h3 > a').click()
    time.sleep(0.5)
    driver.find_element(By.XPATH, '''//*[@id="container"]/div/form/fieldset/div/section/article[1]/article/article[2]/div[1]/div[2]/ul/li[1]/a''').click()
    time.sleep(0.5)
    if gu == '중랑구' or gu == '은평구' or gu == '종로구' or gu == '중구':
        scroll = driver.find_element(By.ID, 'mCSB_2_container')
        driver.execute_script('arguments[0].style.top = "-93px";', scroll)
        time.sleep(0.5)
    driver.find_element(By.LINK_TEXT, gu).click()
    time.sleep(1)

    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    li_elements = soup.select('#mCSB_3_container > ul > li')
    storeNum = len(li_elements)

    gu_of_store, name_of_store, add_of_store, latitude_of_store, longitude_of_store = [], [], [], [], []

    for i in range(storeNum):
        gu_of_store.append(gu)
        name_of_store.append(li_elements[i].get('data-name'))
        add_of_store.append(li_elements[i].find('p').text[:-9]) #9글자의 전화번호는 제외하고 가져옴
        latitude_of_store.append(li_elements[i].get('data-lat'))
        longitude_of_store.append(li_elements[i].get('data-long'))

    shops = [gu_of_store, name_of_store, add_of_store, latitude_of_store, longitude_of_store]
    return shops


In [487]:
df_added = pd.DataFrame()
for i in range(0, len(guList)):
    shop_info = nameNaddress(guList[i])
    df = pd.DataFrame({
        '구': shop_info[0],
        '지점명': shop_info[1],
        '주소' :shop_info[2],
        '위도':shop_info[3],
        '경도':shop_info[4]})
    
    df_added = pd.concat([df_added, df])

In [504]:
starbucks_df = df_added.reset_index(drop=True)

In [505]:
starbucks_df

,구,지점명,주소,위도,경도
0,관악구,봉천역,서울특별시 관악구 남부순환로 1730 (봉천동),37.4820375340064,126.942287792538
1,관악구,서울대입구역8번출구,서울특별시 관악구 남부순환로 1831 (봉천동),37.4812061,126.9536942
2,관악구,서울대입구역,서울특별시 관악구 남부순환로 1812 (봉천동),37.48118,126.951324
3,관악구,신림타임스트림,서울특별시 관악구 신림로 330 타임스트림 1층,37.48385383,126.9301799
4,관악구,신림,서울특별시 관악구 신림로 355-1 (신림동),37.48609493,126.9291602
...,...,...,...,...,...
605,영등포구,여의도IFC몰(L1)R,서울특별시 영등포구 국제금융로 10 (여의도동) 지하1층 178호,37.525127,126.925508
606,영등포구,당산대로,서울특별시 영등포구 양평로 22 (당산동6가),37.533121,126.903804
607,영등포구,영등포역,서울특별시 영등포구 경인로 870 (영등포동1가),37.5169614,126.9091390
608,영등포구,샛강역,서울특별시 영등포구 여의대방로 376 (여의도동),37.5183551,126.9312716


In [506]:
driver.quit()

### 서울시의 이디야커피 매장의 이름과 주소, 구 이름을 pandas data frame으로 정리

In [513]:
# 이디야 페이지 접근
driver = webdriver.Chrome()
driver.get('https://ediya.com/contents/find_store.html#c')
driver.find_element(By.CSS_SELECTOR, '#contentWrap > div.contents > div > div.store_search_pop > ul > li:nth-child(2) > a').click()

- 구별로 작성하여 검색하니 다음과 같은 멘트가 출력됨

      - 검색 결과가 너무 많습니다. 범위를 좁혀 주시기 바랍니다.
- 확인해보니, '중구'와 같은 지역은 서울 지역 외에도 존재하여 다음과 같은 멘트가 출력된다는 것을 확인
- 키워드 앞에 '서울' 을 붙이기로 결정

In [509]:
# guList 변경
for i in range(len(guList)):
    guList[i] = f'서울 {guList[i]}'

guList

['서울 관악구',
 '서울 강북구',
 '서울 광진구',
 '서울 도봉구',
 '서울 구로구',
 '서울 중랑구',
 '서울 강남구',
 '서울 동작구',
 '서울 성동구',
 '서울 노원구',
 '서울 서초구',
 '서울 종로구',
 '서울 송파구',
 '서울 서대문구',
 '서울 용산구',
 '서울 동대문구',
 '서울 강동구',
 '서울 은평구',
 '서울 양천구',
 '서울 강서구',
 '서울 마포구',
 '서울 성북구',
 '서울 금천구',
 '서울 중구',
 '서울 영등포구']

In [514]:
shops_ediya = []
for i in range(len(guList)):
    driver.find_element(By.CSS_SELECTOR, '#keyword').send_keys(guList[i])
    driver.find_element(By.CSS_SELECTOR, '#keyword_div > form > button').click()
    time.sleep(1)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    time.sleep(1)
    ediya = soup.select('#placesList > li')
    time.sleep(1)
    for store in ediya:
        gu = store.select_one('dd').text.split(" ")[1]
        address = store.select_one('dd').text
        store_name = store.select_one('dt').text
        shops_ediya.append({
            "구" : gu,
            "지점명" : store_name,
            "주소" : address})
    driver.find_element(By.CSS_SELECTOR, '#keyword').clear()  # 수정된 부분
    time.sleep(1)

In [524]:
driver.quit()

- 이디야는 웹페이지 내에서 위도 경도를 확인할 수 없어, 구글에서 직접 찾아와야 함

In [515]:
shops_ediya

[{'구': '관악구', '지점명': '관악우성점', '주소': '서울 관악구 관악로 232 (봉천동)'},
 {'구': '관악구', '지점명': '관악하나로마트점', '주소': '서울 관악구 남부순환로 1369 (신림동, 관악농협농산물백화점)'},
 {'구': '관악구', '지점명': '낙성대역점', '주소': '서울 관악구 남부순환로 1915 (봉천동)'},
 {'구': '관악구', '지점명': '난곡사거리점', '주소': '서울 관악구 남부순환로 1444 (신림동)'},
 {'구': '관악구', '지점명': '난향동점', '주소': '서울 관악구 난곡로 60 (신림동)'},
 {'구': '관악구', '지점명': '남현홈플러스점', '주소': '서울 관악구 과천대로 909 (남현동, 남현프라자)'},
 {'구': '관악구', '지점명': '녹두거리점', '주소': '서울 관악구 신림로 91 (신림동)'},
 {'구': '관악구', '지점명': '당곡사거리점', '주소': '서울 관악구 봉천로 245 (봉천동)'},
 {'구': '관악구', '지점명': '보라매중앙점', '주소': '서울 관악구 보라매로 62 (봉천동, 보라매삼성아파트)'},
 {'구': '관악구', '지점명': '봉일시장점', '주소': '서울 관악구 은천로 23 (봉천동)'},
 {'구': '관악구', '지점명': '봉천역점', '주소': '서울 관악구 남부순환로 1725 (봉천동)'},
 {'구': '관악구', '지점명': '봉천중앙점', '주소': '서울 관악구 남부순환로 1874 (봉천동)'},
 {'구': '관악구', '지점명': '봉천현대시장점', '주소': '서울 관악구 양녕로 39-1 (봉천동)'},
 {'구': '관악구', '지점명': '사당중앙점', '주소': '서울 관악구 과천대로 951 (남현동)'},
 {'구': '관악구', '지점명': '서울대중앙점', '주소': '서울 관악구 남부순환로216길 2 (봉천동)'},
 {'구': '관악구', '지점명': '신대방역점'

In [517]:
ediya_df = pd.DataFrame(shops_ediya)
ediya_df.tail()

,구,지점명,주소
730,영등포구,영등포코레일점,"서울 영등포구 경인로 846 (영등포동, 영등포 민자역사)"
731,영등포구,영등포홈플러스점,"서울 영등포구 당산로 42 (문래동3가, 홈플러스)"
732,영등포구,한강성심병원점,서울 영등포구 버드나루로 51 (영등포동2가)
733,영등포구,한국수출입은행점,"서울 영등포구 은행로 38 (여의도동, 한국수출입은행빌딩)"
734,영등포구,힐스테이트클래시안점,"서울 영등포구 신길로28길 9 (신길동, 힐스테이트 클래시안)"


In [519]:
import googlemaps
gmaps_key = '###################' #암호화함
gmaps = googlemaps.Client(key=gmaps_key)

In [520]:
ediya_df['위도'] = None
ediya_df['경도'] = None

In [526]:
for idx, rows in ediya_df.iterrows():
    address = rows['주소']
    address_list = address.split(" ")

    tmp = gmaps.geocode(address, language='ko')
    if tmp:  # 반환된 결과가 있는지 확인
        latitude = tmp[0].get('geometry', {}).get('location', {}).get('lat', None)
        longitude = tmp[0].get('geometry', {}).get('location', {}).get('lng', None)
        if latitude is not None and longitude is not None:  # 위도와 경도가 모두 유효한지 확인
            ediya_df.loc[idx, '위도'] = latitude
            ediya_df.loc[idx, '경도'] = longitude
        else:
            print(f"위도 또는 경도를 찾을 수 없습니다: {address}")
    else:
        ediya_df.loc[idx, '위도'] = None
        ediya_df.loc[idx, '경도'] = None

In [530]:
ediya_df.head()

,구,지점명,주소,위도,경도
0,관악구,관악우성점,서울 관악구 관악로 232 (봉천동),37.484948,126.956027
1,관악구,관악하나로마트점,"서울 관악구 남부순환로 1369 (신림동, 관악농협농산물백화점)",37.480127,126.902257
2,관악구,낙성대역점,서울 관악구 남부순환로 1915 (봉천동),37.477926,126.962222
3,관악구,난곡사거리점,서울 관악구 남부순환로 1444 (신림동),37.480904,126.911275
4,관악구,난향동점,서울 관악구 난곡로 60 (신림동),37.460814,126.919383


In [555]:
ediya_df.shape

(735, 5)

### 지도 시각화

#### 스타벅스 지도 시각화

In [533]:
import json
import folium
import warnings
warnings.simplefilter(action='ignore', category='FutureWarning')

In [535]:
geo_path = '../data/02. skorea_municipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))

In [544]:
starbucks_map = folium.Map(location=[37.5502, 126.982], zoom_start=12, titles='CartoDB positron')

In [545]:
for idx, row in starbucks_df.iterrows():
    latitude = row['위도']
    longitude = row['경도']

    folium.CircleMarker(
        location=[latitude, longitude],
        radius=1.2,
        color='green'
    ).add_to(starbucks_map)
starbucks_map

#### 이디야 지도 시각화

In [549]:
ediya_map = folium.Map(location=[37.5502, 126.982], zoom_start=12, titles='CartoDB positron')

for idx, row in ediya_df.iterrows():
    latitude = row['위도']
    longitude = row['경도']

    if latitude is not None and longitude is not None:
        folium.CircleMarker(
            location=[latitude, longitude],
            radius=1.2,
            color='blue'
    ).add_to(ediya_map)
ediya_map

### 지도 합치기

In [553]:
result_map = folium.Map(location=[37.5502, 126.982], zoom_start=12, tiles='CartoDB positron')

for idx, row in starbucks_df.iterrows():
    latitude = row['위도']
    longitude = row['경도']

    folium.CircleMarker(
        location=[latitude, longitude],
        radius=1.5,
        color='green',
        legend_name='starbucks'
    ).add_to(result_map)

for idx, row in ediya_df.iterrows():
    latitude = row['위도']
    longitude = row['경도']

    if latitude is not None and longitude is not None:
        folium.CircleMarker(
            location=[latitude, longitude],
            radius=1.5,
            color='blue',
            legend_name='ediya'
    ).add_to(result_map)

result_map

#### 분석 결과 해석
가설: 이디야는 스타벅스 커피 매장이 위치하는 곳에 매장을 위치시키는 전략을 채택한다.   
해석: **스타벅스 커피 매장 근처에 이디야가 위치해 있다는 것은 일부 사실이나, 이것을 기업의 전략으로 채택했다거나 의도했다고 보기는 어렵다.**  
- 만약 이디야가 스타벅스 커피 매장 근처에 매장을 위치시킨다는 전략을 채택하고 있었다면, 스타벅스 지점의 개수가 월등히 많은 강남에서 스타벅스와 이디야의 비중 혹은 개수가 비슷할 것이다. 하지만 아래에서 확인할 수 있듯이 그렇지 않다.

In [584]:
print('강남 스타벅스 개수:',len(starbucks_df[starbucks_df['구']=='강남구']))

강남 스타벅스 개수: 90


In [585]:
print('강남 이디야 개수:',len(ediya_df[ediya_df['구']=='강남구']))

강남 이디야 개수: 45


In [572]:
gangnam_ed = len(ediya_df[ediya_df['구']=='강남구']) / len(ediya_df) * 100
gangnam_sb = len(starbucks_df[starbucks_df['구']=='강남구']) / len(starbucks_df) * 100
print(f'스타벅스의 강남구 비중: {round(gangnam_sb,2)}\n이디야의 강남구 비중: {round(gangnam_ed,2)}')

스타벅스의 강남구 비중: 14.75
이디야의 강남구 비중: 6.12


- 뿐만 아니라, 구로구와 중랑구 특히 중랑구는 스타벅스의 수보다 이디야 지점의 수가 더 많다.
- 구로구는 지도에서 확인할 수 있듯이 이디야의 입점 위치가 스타벅스와 큰 관련이 없고, 중랑구는 위치하고 있는 비중에서 꽤나 차이를 보인다. 

In [573]:
guro_ed = len(ediya_df[ediya_df['구']=='구로구']) / len(ediya_df) * 100
guro_sb = len(starbucks_df[starbucks_df['구']=='구로구']) / len(starbucks_df) * 100
print(f'스타벅스의 구로구 비중: {round(guro_sb,2)}\n이디야의 구로구 비중: {round(guro_ed,2)}')

스타벅스의 구로구 비중: 2.3
이디야의 구로구 비중: 3.95


In [574]:
jungnang_ed = len(ediya_df[ediya_df['구']=='중랑구']) / len(ediya_df) * 100
jungnang_sb = len(starbucks_df[starbucks_df['구']=='중랑구']) / len(starbucks_df) * 100
print(f'스타벅스의 중랑구 비중: {round(jungnang_sb,2)}\n이디야의 중랑구 비중: {round(jungnang_ed,2)}')

스타벅스의 중랑구 비중: 1.31
이디야의 중랑구 비중: 4.08


따라서, 개인적으로 이디야가 스타벅스 주위에 입점하는 전략을 채택하고 있다는 가설은 참이 아니라고 생각한다.